# Aquí inicia la base buena

In [1]:
from datetime import datetime , date
import mysql.connector
import pandas as pd 


In [2]:
mydb = mysql.connector.connect(
  host='localhost',
  port='3306',
  user='root',
  password='password'
)

mycursor = mydb.cursor()

mycursor.execute("DROP DATABASE IF EXISTS panaderia")
mycursor.execute("CREATE DATABASE  panaderia")

mycursor.execute("SHOW DATABASES")

for x in mycursor:
  print(x)

('information_schema',)
('mysql',)
('panaderia',)
('performance_schema',)
('sys',)


In [3]:
mydb = mysql.connector.connect(
  host='localhost',
  port='3306',
  user='root',
  password='password',
  database="panaderia"
)

mycursor = mydb.cursor()

In [4]:
mycursor.execute("""
                    CREATE TABLE productos(
                        id int(3) NOT NULL AUTO_INCREMENT, 
                        nombre varchar(30), 
                        precio_unit double, 
                        descripcion text(500), 
                        categoria varchar(30), 
                        cantidad_disp int, 
                        PRIMARY KEY(id)
                    )
                """)

In [5]:
mycursor.execute("""
                    CREATE TABLE trabajadores(
                        RFC varchar(20) NOT NULL,
                        es_admin int(1) DEFAULT 0,  
                        nombre varchar(30), 
                        apellidos varchar(60), 
                        contraseña varchar(50),
                        PRIMARY KEY(RFC)
                    )
                """)

In [6]:
mycursor.execute("""
                    CREATE TABLE hist_gastos(
                        fecha datetime, 
                        concepto text(500),
                        costo    double, 
                        usuario  varchar(20) NOT NULL,
                        FOREIGN KEY(usuario) REFERENCES trabajadores(RFC) ON DELETE CASCADE,
                        PRIMARY KEY(fecha)
                    )
                """)

In [7]:
mycursor.execute("""
                    CREATE TABLE hist_ingresos_extras(
                        fecha datetime, 
                        concepto text(500),
                        ingreso    double, 
                        usuario  varchar(20) NOT NULL,
                        FOREIGN KEY(usuario) REFERENCES trabajadores(RFC) ON DELETE CASCADE,
                        PRIMARY KEY(fecha)
                    )
                """)

In [8]:
mycursor.execute("""
                    CREATE TABLE reportes(
                        dia date,
                        gastos_extras double, 
                        ingresos_extras double, 
                        ingresos_tienda double, 
                        ingresos_total double, 
                        ganancias_virtual double, 
                        ganancias_real double, 
                        perdidas_prop double,
                        PRIMARY KEY(dia)
                    )
                """)

In [9]:
mycursor.execute("SHOW TABLES")

for x in mycursor:
  print(x)

('hist_gastos',)
('hist_ingresos_extras',)
('productos',)
('reportes',)
('trabajadores',)


## Consultas

In [10]:
def existe_producto(nombre : str): 
    mycursor.execute(
        """
            SELECT * FROM productos
            WHERE nombre = %s
            LIMIT 1
        """, 
        [nombre]
    )
    myresult = mycursor.fetchone()

    if myresult: 
        return True 
    return False

In [11]:
def nuevo_producto(nombre: str, precio_unit: float, descripcion: str, categoria: str, cantidad_disp: int):    
    sql = "INSERT INTO productos (nombre, precio_unit, descripcion, categoria, cantidad_disp) VALUES (%s, %s, %s, %s, %s)"
    val = [nombre, precio_unit, descripcion, categoria, cantidad_disp]
    mycursor.execute(sql, val)
    return 
    


In [12]:
def update_cant_producto(nombre: str, cantidad_disp: int): 
    sql = "UPDATE productos SET cantidad_disp = %s WHERE nombre = %s"
    val = [cantidad_disp, nombre]
    mycursor.execute(sql, val)
    return

In [13]:
def get_all_productos(): 
    result = pd.read_sql("SELECT nombre, precio_unit, cantidad_disp FROM productos", mydb)
    return result

In [14]:
def existe_usuario( rfc: str):
    mycursor.execute(
        """
            SELECT * FROM trabajadores
            WHERE nombre = %s
            LIMIT 1
        """, 
        [rfc]
    )
    myresult = mycursor.fetchone()

    if myresult: 
        return True 
    return False 


In [15]:
def nuevo_usuario( rfc: str, es_admin: int, nombre: str, apellidos: str, contraseña: str):
    sql = "INSERT INTO trabajadores (rfc, es_admin, nombre, apellidos, contraseña) VALUES (%s, %s, %s, %s, %s)"
    val = [rfc, es_admin, nombre, apellidos, contraseña]
    mycursor.execute(sql, val)
    return 


In [16]:
def guardar_gasto(fecha: datetime, concepto: str, costo: float, usuario: str): 
    sql = "INSERT INTO hist_gastos (fecha, concepto, costo, usuario) VALUES (%s, %s, %s, %s)"
    val = [fecha, concepto, costo, usuario]
    mycursor.execute(sql, val)
    return 

In [17]:
def guardar_ingreso(fecha: datetime, concepto: str, ingreso: float, usuario: str): 
    sql = "INSERT INTO hist_ingresos_extras (fecha, concepto, ingreso, usuario) VALUES (%s, %s, %s, %s)"
    val = [fecha, concepto, ingreso, usuario]
    mycursor.execute(sql, val)
    return 

In [18]:
def guardar_reporte(dia: date, gastos_extras: float, 
                    ingresos_extras: float, ingresos_tienda: float, ingresos_total: float, 
                    ganancias_virtual: float, ganancias_real: float, perdidas_prop: float):
    sql = """
            INSERT INTO reportes VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
          """
    val = [dia, gastos_extras, ingresos_extras, ingresos_tienda, ingresos_total, 
           ganancias_virtual, ganancias_real, perdidas_prop]
    mycursor.execute(sql, val)
    return 

# Cargando Datos 

In [19]:
nuevo_usuario("LAMY0004069N5", 1, "Yeudiel", "Lara Moreno", "12345")

In [20]:
guardar_reporte(date.today(), 3200, 500, 4000, 4500, 1300, 1100, 0.16)

In [21]:
df_productos = pd.read_csv('data\productos.csv')
df_productos.head()

,nombre,precio_unit,descripcion,categoria,cantidad_disp
0,Volovan Jamon,15,"Hojaldre 20x10 cm, relleno con jamon, queso am...",volovan salado,10
1,Volovan Pollo,15,"Hojaldre 20x10 cm,relleno con pollo, tomate, c...",volovan salado,10
2,Volovan Choriqueso,15,"Hojaldre 20x10 cm, relleno con chorizo, queso ...",volovan salado,10
3,Volovan Tres quesos,15,"Hojaldre 20x10 cm, relleno con queso manchego,...",volovan salado,10
4,Volovan Champiñones,15,"Hojaldre 20x10 cm, relleno con champiñones, qu...",volovan salado,10


In [22]:
#Cargamos nuestros datos
for index, row in df_productos.iterrows():
        nuevo_producto(row[0], row[1], row[2], row[3], row[4])

In [23]:
update_cant_producto("Volovan Jamon", 20)

In [24]:
get_all_productos()

,nombre,precio_unit,cantidad_disp
0,Volovan Jamon,15.0,20
1,Volovan Pollo,15.0,10
2,Volovan Choriqueso,15.0,10
3,Volovan Tres quesos,15.0,10
4,Volovan Champiñones,15.0,10
5,Volovan Hawaiano,15.0,10
6,Volovan Atún,15.0,10
7,Volovan Pastor,18.0,10
8,Volovan Nutella,15.0,10
9,Volovan Zarzamora,15.0,10
